Create stored procedure for master time table.

In [ ]:
CREATE OR ALTER PROCEDURE [master].[Initialize_TimeTable]
AS
BEGIN
	SET NOCOUNT ON;

    DROP TABLE IF EXISTS [master].[Time];

    WITH h AS (
        SELECT TOP 24 rn = ROW_NUMBER() OVER (ORDER BY s1.[object_id]) - 1
        FROM sys.all_objects AS s1
            CROSS JOIN sys.all_objects AS s2
        ORDER BY s1.[object_id]
    ),
    m AS (
        SELECT TOP 60 rn = ROW_NUMBER() OVER (ORDER BY s1.[object_id]) - 1
        FROM sys.all_objects AS s1
            CROSS JOIN sys.all_objects AS s2
        ORDER BY s1.[object_id]
    )

    SELECT
        [time_key]
        , CONVERT(NVARCHAR, [time_key], 8) AS hhmmss
        , LEFT(CONVERT(NVARCHAR, [time_key], 8), 5) AS hhmm
        , [hour]
        , [minute]
        , [second]
        , IIF([hour] < 12, 'AM', 'PM') AS [period_of_day]
    INTO [master].[Time]
    FROM (
        SELECT
            CAST(CONCAT(h.rn, ':', m.rn, ':', s.rn) AS TIME) AS [time_key]
            , h.rn AS [hour]
            , m.rn AS [minute]
            , s.rn AS [second]
        FROM h
            CROSS APPLY m 
            CROSS APPLY m AS s
    ) t
    ORDER BY [time_key];
        
END
GO

Run the stored procedure to create new [master].[Time] table.

In [ ]:
EXEC [master].[Initialize_TimeTable];
GO